In [45]:
import os

xx=os.getcwd()
import pathlib
q=pathlib.Path(xx)
jj=str(q.parent)
jj

'/home/daisy/mydrive/web-dev/deep_learning_pipeline-train-inference-/training_controller'

In [46]:
label_map_path='/post_data/fc3f4e42-c92c-11eb-8ce8-47e7b178684d/Letters_label_map.pbtxt'

In [47]:
def read_label_map(label_map_path):

    item_id = None
    item_name = None
    items = {}
    label_map_path=jj+label_map_path
    with open(label_map_path, "r") as file:
        for line in file:
            
            line.replace(" ", "")
            if line == "item{":
                pass
            elif line == "}":
                pass
            elif "id" in line:
                item_id = line.split(":", 1)[1].strip()
            elif "name" in line:
                item_name = line.split(":", 1)[1].replace("'", "").strip()

            if item_id is not None and item_name is not None:
                items[item_name] = item_id
                item_id = None
                item_name = None

    return items



In [48]:
m=read_label_map(label_map_path)
len(m)

26

In [49]:
import pandas as pd
import json
config_file_path='/home/daisy/Downloads/amercan_sgn/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.config'
final_config_file_path='/home/daisy/Downloads/amercan_sgn/pipeline_config_final.config'
# label_path='/home/daisy/Downloads/amercan_sgn/train/Letters_label_map.pbtxt'
checkpoint_path='apis/tf_base_od_models/faster_rcnn_resnet101_v1_640x640_coco17_tpu-8/checkpoint/ckpt-0'
train_record_file='/home/daisy/Downloads/amercan_sgn/train/Letters.tfrecord'
test_record_file='/home/daisy/Downloads/amercan_sgn/train/valid_letters.tfrecord'

In [52]:

import argparse

import tensorflow as tf
from google.protobuf import text_format
from object_detection.protos import pipeline_pb2

def configue_the_pipeline_file_tf_faster_rcnn(config_file_path,label_path,checkpoint_path,train_record_file,test_record_file)->str:
    pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
    
    with tf.io.gfile.GFile(config_file_path, "r") as f:                                                                                                                                                                                                                     
        proto_str = f.read()                                                                                                                                                                                                                                          
        text_format.Merge(proto_str, pipeline_config)
    classes_dict=read_label_map(label_path)
    
    pipeline_config.model.faster_rcnn.num_classes=len(classes_dict)
    pipeline_config.train_config.fine_tune_checkpoint=checkpoint_path
    pipeline_config.train_config.fine_tune_checkpoint_type='detection'
    pipeline_config.train_config.batch_size=int(1)
    
    
    pipeline_config.train_input_reader.label_map_path=label_path
    pipeline_config.train_input_reader.tf_record_input_reader.input_path[:]=[train_record_file]
    pipeline_config.eval_input_reader[0].label_map_path=label_path
    pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:]=[test_record_file]

   
    config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
    with tf.io.gfile.GFile(final_config_file_path, "wb") as f:                                                                                                                                                                                                                       
        f.write(config_text)
    
    print(config_text)
    
    return final_config_file_path

In [53]:
final_path=configue_the_pipeline_file_tf_faster_rcnn(config_file_path,label_map_path,checkpoint_path,train_record_file,test_record_file)

model {
  faster_rcnn {
    num_classes: 26
    image_resizer {
      keep_aspect_ratio_resizer {
        min_dimension: 640
        max_dimension: 640
        pad_to_max_dimension: true
      }
    }
    feature_extractor {
      type: "faster_rcnn_resnet50_keras"
      batch_norm_trainable: true
    }
    first_stage_anchor_generator {
      grid_anchor_generator {
        height_stride: 16
        width_stride: 16
        scales: 0.25
        scales: 0.5
        scales: 1.0
        scales: 2.0
        aspect_ratios: 0.5
        aspect_ratios: 1.0
        aspect_ratios: 2.0
      }
    }
    first_stage_box_predictor_conv_hyperparams {
      op: CONV
      regularizer {
        l2_regularizer {
          weight: 0.0
        }
      }
      initializer {
        truncated_normal_initializer {
          stddev: 0.01
        }
      }
    }
    first_stage_nms_score_threshold: 0.0
    first_stage_nms_iou_threshold: 0.7
    first_stage_max_proposals: 300
    first_stage_localization_loss

In [10]:
final_path

'/home/daisy/Downloads/amercan_sgn/pipeline_config_final.config'

In [ ]:
pipeline_config.train_config.batch_size=int(2)

In [ ]:
pipeline_config

In [ ]:
configue_the_pipeline_file_tf_faster_rcnn(file_path)

In [ ]:
import argparse

import tensorflow as tf
from google.protobuf import text_format
from object_detection.protos import pipeline_pb2


pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
print(pipeline_config)
with tf.io.gfile.GFile(file_path, "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config) 
#pipeline_config.train_input_config.label_map_path= "/home/daisy/Downloads/amercan_sgn/train/Letters_label_map.pbtxt"

config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(path_2, "wb") as f:                                                                                                                                                                                                                       
    f.write(config_text)  

In [ ]:
pipeline_config

In [ ]:
items=read_label_map("/home/daisy/Downloads/amercan_sgn/train/Letters_label_map.pbtxt")

pipeline_config.model.faster_rcnn.num_classes=len(items)


In [ ]:
pipeline_config.model

In [ ]:
pipeline_config

In [ ]:
def get_image_resizer_config(model_config):
  """Returns the image resizer config from a model config.
  Args:
    model_config: A model_pb2.DetectionModel.
  Returns:
    An image_resizer_pb2.ImageResizer.
  Raises:
    ValueError: If the model type is not recognized.
  """
  meta_architecture = model_config.WhichOneof("model")
  meta_architecture_config = getattr(model_config, meta_architecture)

  if hasattr(meta_architecture_config, "image_resizer"):
    return getattr(meta_architecture_config, "image_resizer")
  else:
    raise ValueError("{} has no image_reszier_config".format(
        meta_architecture))

In [ ]:
get_image_resizer_config(pipeline_config)

In [ ]:
meta_architecture = model_config.WhichOneof("model")

In [ ]:
type(pipeline_config)


In [ ]:
pipeline_config.train_confi1g.fine_tune_checkpoint

In [ ]:
pipeline_config.train_config.fine_tune_checkpoint_type

In [ ]:
pipeline_config.train_config.batch_size

In [ ]:
pipeline_config.train_input_reader.label_map_path

In [ ]:
pipeline_config.train_input_reader.tf_record_input_reader.input_path

In [ ]:
x="/post_data/0eaed71a-c91d-11eb-a2e9-2ff7dc84e18f/Letters_label_map.pbtxt"

In [ ]:
new_path='/'+x.split('/')[1]+'/'+x.split('/')[2]+'/'

In [ ]:
# new_path='/'+x[1]+'/'+x[2]+'/'

In [ ]:
new_path

In [ ]:
info_dict={'id': 125, 'usecase_type': 'Object detection', 'framework_type': 'tensorflow', 'model_arch_type': 'fasterrcnn', 'training_data': '/post_data/5bce4f28-c92a-11eb-8ce8-47e7b178684d/Letters.tfrecord', 'test_data': '/post_data/5bce4f28-c92a-11eb-8ce8-47e7b178684d/valid_letters.tfrecord', 'label_file_data': '/post_data/5bce4f28-c92a-11eb-8ce8-47e7b178684d/Letters_label_map.pbtxt'}


In [ ]:
info_dict['label_file_data']